In [104]:
## Read PDF files
import nest_asyncio

nest_asyncio.apply()
#GET LLAMA_CLOUD_API_KEY
import os
from llama_parse import LlamaParse

api_key: str | None = os.getenv("LLAMA_CLOUD_API_KEY")

parsing_instructions = "Do not take into account the page breaks (no --- between pages), do not repeat the header and the footer so the tables are merged. Keep the same format for similar tables."
#parsing_instructions = "Keep the same format for similar tables. Avoid repetitions. Don't mark the page breaks."

parser = LlamaParse(
    api_key= api_key,  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown",  # "markdown" and "text" are available
    gpt4o_mode=True,
    verbose=True,
    language="fr",  # Optionally you can define a language, default=en
    parsing_instruction=parsing_instructions,  # Optionally you can define a parsing instruction
)
# sync
documents = parser.load_data("./docs/CHARTE_QUALITE_PRODUIT2.pdf")
print(documents)


Started parsing the file under job_id 10b2ac0f-b790-406a-b850-77274fff4f51
.[Document(id_='29582cca-cdc7-4479-a795-967898e13265', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="# CHARTE PRODUITS COUP DE PATES\n\n**Codification :**  \nCDP_QUA_CHART_01\n\n**Version :** 5  \n**Date d’application :** 13/12/2023\n\n| Date       | Mises à jour                                                                 |\n|------------|------------------------------------------------------------------------------|\n| 19/12/2014 | Création                                                                     |\n| 12/12/2019 | Insertion des additifs interdits et à éviter                                 |\n| 13/05/2022 | Revue des exigences recettes et annexes                                      |\n| 30/03/2023 | Revue des annexes I et II. Fréquence de mise à jour CDC                      |\n| 13/12/2023 | Ajout d’une exigence de certifica

In [103]:
with open('./docs/CHARTE_QUALITE_PRODUIT_gpt4o.md', "w") as md_file:
    md_file.write(documents[0].get_content())


### Parse PDF usinf pdf2docx 

In [ ]:
from pdf2docx import Converter
pdf_file = "./docs/rib-chloe.pdf"
cv = Converter(pdf_file)
cv.convert() #type: ignore
cv.close()

In [ ]:
# from docx import Document

# def formatTable(table):
#     formatted_table = []
#     for row in table.rows:
#         list_row = []
#         for cell in row.cells:
#             for paragraph in cell.paragraphs:
#                 list_row.append(paragraph.text)
#         formatted_table.append(tuple(list_row))
#     return tuple(formatted_table)

In [ ]:
# document = Document('./docs/CHARTE_QUALITE_PRODUIT2.docx')
# unique_tables = set()
# pruned_tables = []
# for i,table in enumerate(document.tables):
#     formatted_table = formatTable(table)
#     if formatted_table not in unique_tables:
#         pruned_tables.append(formatted_table)
#         unique_tables.add(formatted_table)
#     else:
#         print("Duplicate table found")
#         # Remove the table's associated XML elements
#         table._element.getparent().remove(table._element)

# print(pruned_tables)
#document.save('./docs/MODIFIED_CHARTE_QUALITE.docx')

In [ ]:
import mammoth
from markdownify import markdownify as md

with open('./docs/CHARTE_QUALITE_PRODUIT2.docx', "rb") as docx_file:
    result = mammoth.convert_to_html(docx_file)

with open('./docs/CHARTE_QUALITE_PRODUIT2.html', "rb") as html_file:
    html_file.write(result.value)

md_result = md(result.value)
#save markdown to file
with open('./docs/CHARTE_QUALITE_PRODUIT2.md', "w") as md_file:
    md_file.write(md_result)

print(f"DOC LENGTH : {len(md_result)} characters")



In [ ]:
from bs4 import BeautifulSoup

def remove_duplicate_tables(html_file):
    with open(html_file, 'r') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all tables
    tables = soup.find_all('table')

    # Keep track of tables that have appeared
    seen_tables = set()

    # Iterate over tables in reverse order to delete duplicate occurrences
    for table in tables:
        # Convert the table to a string for comparison
        table_str = str(table)
        # Check if table has appeared before
        if table_str in seen_tables:
            table.extract()  # Remove duplicate table
        else:
            seen_tables.add(table_str)
    return soup

def remove_page_status(soup):

    #soup = BeautifulSoup(html_content, 'html.parser')

    # Find all elements containing the text "Page"
    page_elements = soup.find_all(text=lambda text: 'Page' in text)

    # Remove the parent element of each page element
    for page_element in page_elements:
        parent = page_element.find_parent()
        parent.extract()

    return soup

def merge_adjacent_tables(soup):
    """
    Merge adjacent tables with the same number of columns and unnest nested tables in the HTML content.

    Args:
    soup (BeautifulSoup): The BeautifulSoup object representing the parsed HTML.

    Returns:
    BeautifulSoup: The modified BeautifulSoup object with merged tables and unnested tables.
    """

    # Find all tables
    tables = soup.find_all('table')

    # Unnest nested tables
    for table in tables:
        nested_tables = table.find_all('table')
        if nested_tables:
            for nested_table in nested_tables:
                # Move the nested table after the outer table
                table.insert_after(nested_table.extract())
            table.extract()  # Remove the outer table

    # Find all tables again after unnesting
    tables = soup.find_all('table')

    # Merge adjacent tables with the same number of columns
    i = 0
    while i < len(tables) - 1:
        current_table = tables[i]
        next_table = tables[i + 1]

        # Check if the tables are directly adjacent
        if current_table.next_sibling == next_table:
            # Check if both tables have the same number of columns
            current_table_cols = len(current_table.find_all('tr')[0].find_all(['th', 'td']))
            next_table_cols = len(next_table.find_all('tr')[0].find_all(['th', 'td']))
            if current_table_cols == next_table_cols:
                # Merge the tables
                current_table.extend(next_table.find_all('tr'))
                next_table.extract()
                tables.pop(i + 1)  # Remove the merged table from the list
                continue

        i += 1

    return soup


In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

class TableValidator(BaseModel):
    description: str = Field(description="Description of the table content")
    to_change: str = Field(description="Explanation of invalidity and changes to be made to the table content if it is invalid")
    is_valid: bool = Field(description="Validation status of the table")
    correction: str = Field(description="Correction of the input html table")

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

parser = PydanticOutputParser(pydantic_object= TableValidator)
template = """You are an HTML table cleaning bot. Your task is to analyze the provided HTML content and determine if it is a valid table.

If you determine that the HTML is not a valid table (i.e., the content doesn't fit the structure or semantics of a table), you should modify the HTML by removing the table tags and converting the content into a more appropriate HTML structure, such as paragraphs or lists.
Do not force change if it is readable like this.\n
If everything is correct but a line, modify the line to make it correct, do not delete it. Think about the homogeneity of the table.\n
In description you should provide a brief description of the table and if incorrect, a clear explaination of why it is incorrect and how it may impact the reader comprehension.\n
Note that is_valid cannot be true if at least one element is not correctly formatted.\n
If it looks like a text that could me simply put in a paragraph do it.\n
If you determine that the HTML is a normal table and doesn't require any modifications, you should return the entire original HTML in the output.\n
{format_instructions}\n
Return only the JSON object.
Here is the table:
{table}"""

prompt = PromptTemplate(
    template=template,
    input_variables=["table"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
llm = ChatOpenAI(model="gpt-4o", temperature = 0.0)
llm_chain = prompt | llm | parser

In [ ]:
import tqdm
def process_tables(soup):
    #get all tables
    tables = soup.find_all('table')
    #for each table send to OpenAI and ask to format
    for table in tqdm.tqdm(tables):
        table_str = str(table)
        if len(table_str) < 10000:
            formatted_table : TableValidator = llm_chain.invoke({"table" : table_str})
            #if table is not valid, replace the table with the formatted table
            if not formatted_table.is_valid:
                table.replace_with(BeautifulSoup(formatted_table.correction, 'html.parser'))
    
    return soup


In [ ]:
# Usage
result = remove_duplicate_tables('./docs/CHARTE_QUALITE_PRODUIT2.html')
result = remove_page_status(result)
result = merge_adjacent_tables(result)


In [ ]:
result = process_tables(result)

In [ ]:
md_result = md(str(result))
#save html to file

with open('./docs/CHARTE_QUALITE_PRODUIT2_Processed.html', "w") as html_file:
    html_file.write(str(result))
#save markdown to file
with open('./docs/CHARTE_QUALITE_PRODUIT2_Processed.md', "w") as md_file:
    md_file.write(md_result)

### Parse PDF using UnstructuredPDFLoader

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader

loader = UnstructuredPDFLoader("./docs/CHARTE_QUALITE_PRODUIT2.pdf", strategy="hi_res")
doc = loader.load()


In [ ]:
from typing import List

def summary(docs):
    print(f"There are {len(docs)} documents")
    print(f"There are {sum([len(doc.page_content) for doc in docs])} characters in the documents")
    print()
    print(f"Preview : n {docs[0].page_content[:100]}")

summary(doc)


In [ ]:
with open('./docs/CHARTE_QUALITE_PRODUIT2.txt', "w") as txt_file:
    txt_file.write(doc[0].page_content)

### Post Process the loaded content

In [15]:
import re
# function to Delete elements of more then 30 char that repeats itself in the text
def remove_headers_footers(ocr_text, min_length=40, min_repetitions=3):
    segments = ocr_text.split("\n")  # Split OCR text into segments (assuming newline-separated)
    repeated_segments = {}  # Dictionary to store repeated segments and their counts

    # Count repetitions of each segment
    for segment in segments:
        if len(segment) >= min_length and re.match(r'^\|( --- \|)*$', segment) is None:
            if segment in repeated_segments:
                repeated_segments[segment] += 1
            else:
                repeated_segments[segment] = 1

    # Identify segments to remove based on repetition count
    segments_to_remove = []
    for segment, count in repeated_segments.items():
        if count >= min_repetitions:
            segments_to_remove.append(segment)

    print(segments_to_remove)
    # Remove identified segments from OCR text
    cleaned_text = ocr_text
    for segment in segments_to_remove:
        cleaned_text = cleaned_text.replace(segment, "")

    return cleaned_text # Strip leading and trailing whitespace





In [49]:
with open("./docs/CHARTE_QUALITE_PRODUIT_gpt4o.md", "r") as text_file:
    md_result = text_file.read()

In [50]:
#md_result = documents[0].get_content()
cleaned_text = remove_headers_footers(md_result, min_length=5, min_repetitions=3)
#cleaned_text = documents[0].text

['# CHARTE PRODUITS COUP DE PATES', '**Codification**: CDP_QUA_CHART_01  ', '**Version**: 5  ', '**Date d’application**: 13/12/2023']


In [51]:
import re
cleaned_text = re.sub(r"^Page.*$", "\n", cleaned_text, flags=re.MULTILINE)#delete the entire line that start with "page .."
cleaned_text =  re.sub(r"\n\n+", "\n\n", cleaned_text) #delete multiple new lines
cleaned_text =  re.sub(r"\n---\n","\n", cleaned_text) #delete the "---" that are present in the text (page jump)
cleaned_text =  re.sub(r"\|\n+\|","|\n|", cleaned_text) #delete spaces between tables

print(f"Cleaned text LENGTH : {len(cleaned_text)} characters")
print(cleaned_text[0:1000])

Cleaned text LENGTH : 35542 characters


**Codification :**  
CDP_QUA_CHART_01

**Version :** 5

**Date d’application :** 13/12/2023

| Date       | Mises à jour                                                                 |
|------------|------------------------------------------------------------------------------|
| 19/12/2014 | Création                                                                      |
| 12/12/2019 | Insertion des additifs interdits et à éviter                                  |
| 13/05/2022 | Revue des exigences recettes et annexes                                       |
| 30/03/2023 | Revue des annexes I et II. Fréquence de mise à jour CDC                       |
| 13/12/2023 | Ajout d’une exigence de certification sur le cacao                           |
|            | Revue des exigences de certification de l’huile de palme                      |

## Table des matières

1. **Exigence recette** .............................................................

In [52]:
with open("./docs/CHARTE_QUALITE_PRODUIT_non_traite.md", "w") as text_file:
    if type(cleaned_text) == str:
        text_file.write(cleaned_text)

In [ ]:
# # Ask an llm to rewrite the text in a more readable & understandable way
# from langchain_core.prompts import PromptTemplate
# from langchain_openai import ChatOpenAI

# template = """You are a markdown file reformater. Only touch markdown tags so it is more readable. Note that title, subtitles and annexes must have a '#', '##', .. Do NOT change the content (do not change or add titles) nor the order of the elements, everything must stay in the file. Keep the language of the sources provided (french). Try to think intelligently on the layout.\n Format the following source: {text} \n\n"""

# prompt = PromptTemplate.from_template(template)

# llm = ChatOpenAI(model="gpt-4-turbo", temperature = 0.0)

# llm_chain = prompt | llm


In [ ]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=5000,
#     chunk_overlap=20,
#     length_function=len,
#     is_separator_regex=True,
#     keep_separator=False,
#     separators=[
        
#         r'\|\n([^|])',
#         "\n\n",
#         "\n",
#         " ",
#     ],
# )
# split_text = text_splitter.split_text(cleaned_text)
# split_text

In [ ]:
# import tqdm
# llm_cleaned_text = ""
# for chunk in tqdm.tqdm(split_text):
#     llm_cleaned_chunk = llm_chain.invoke(chunk)
#     if type(llm_cleaned_chunk.content) == str:
#         llm_cleaned_text += llm_cleaned_chunk.content

# with open("./docs/CHARTE_QUALITE_PRODUIT.md", "w") as text_file:
#     if type(llm_cleaned_text) == str:
#         text_file.write(llm_cleaned_text)


### Use of Langchain to chunk correctly the data

In [53]:
from langchain.text_splitter import MarkdownHeaderTextSplitter
with open("./docs/CHARTE_QUALITE_PRODUIT_non_traite.md", "r") as text_file:
    markdown_text = text_file.read()
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[("**", "Header 1")])
md_header_splits = markdown_splitter.split_text(markdown_text)
md_header_splits


[Document(page_content="**Codification :**\nCDP_QUA_CHART_01  \n**Version :** 5  \n**Date d’application :** 13/12/2023  \n| Date       | Mises à jour                                                                 |\n|------------|------------------------------------------------------------------------------|\n| 19/12/2014 | Création                                                                      |\n| 12/12/2019 | Insertion des additifs interdits et à éviter                                  |\n| 13/05/2022 | Revue des exigences recettes et annexes                                       |\n| 30/03/2023 | Revue des annexes I et II. Fréquence de mise à jour CDC                       |\n| 13/12/2023 | Ajout d’une exigence de certification sur le cacao                           |\n|            | Revue des exigences de certification de l’huile de palme                      |  \n## Table des matières  \n1. **Exigence recette** ..............................................................

In [54]:
len(md_header_splits)

1

### Llama index Joint Tabular/Semantic QA

In [105]:
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

In [24]:
from llama_index.readers.file import MarkdownReader
from pathlib import Path

reader = MarkdownReader()
charte = reader.load_data(Path("./docs/CHARTE_QUALITE_PRODUIT2.md"))

In [107]:
#from llama_index.core.node_parser import MarkdownElementNodeParser
from markdown_element import MarkdownElementNodeParser

node_parser = MarkdownElementNodeParser()

In [18]:
import asyncio
import nest_asyncio
import uvloop

if not isinstance(asyncio.get_event_loop(), uvloop.Loop):
    nest_asyncio.apply()

In [28]:
import os
import pickle

if not os.path.exists("./docs/charte.pkl"):
    raw_nodes_charte = node_parser.get_nodes_from_documents(charte, show_progress= False)
    pickle.dump(raw_nodes_charte, open("./docs/charte.pkl", "wb"))
else:
    raw_nodes_charte = pickle.load(open("./docs/charte.pkl", "rb"))

In [29]:
base_nodes_charte, node_mappings_charte = node_parser.get_base_nodes_and_mappings(
    raw_nodes_charte
)

In [7]:
from llama_index.core.schema import IndexNode, TextNode, MetadataMode

example_index_node = [b for b in base_nodes_charte if isinstance(b, TextNode)][
    16
]
example_index_node.get_content()

'6.2. Suivi nutritionnel\n\nLe fournisseur doit communiquer à Coup de Pates une analyse nutritionnelle réalisée par un laboratoire accrédité COFRAC ou équivalent du COFRAC reconnu par l’ILAC dans les pays concernés. Cette analyse doit être réalisée pour chaque nouveau produit référencé, afin de répondre aux exigences d’étiquetage européennes (avec quantification des acides gras trans et des fibres), et à chaque modification de matières premières et/ou de recette. La communication d’analyses nutritionnelles calculées à l’aide d’un logiciel consolidé est également acceptée.\n\nSur demande de Coup de Pates, un nouveau bulletin d’analyse devra être communiqué.'

In [8]:
from llama_index.core.schema import IndexNode, TextNode, MetadataMode
example_index_node = [b for b in base_nodes_charte if isinstance(b, IndexNode)][
    4
]

# Index Node
print(
    f"\n--------\n{example_index_node.get_content(metadata_mode=MetadataMode.ALL)}\n--------\n"
)
# Index Node ID
print(f"\n--------\nIndex ID: {example_index_node.index_id}\n--------\n")
# Referenceed Table
print(
    f"\n--------\n{node_mappings_charte[example_index_node.index_id].get_content()}\n--------\n"
)


--------
col_schema: Column: Additif
Type: string
Summary: None

Column: Code
Type: string
Summary: None

The table lists various food additives along with their corresponding codes, specifically highlighting those with potential carcinogenicity or involvement in serious diseases.,
with the following table title:
ANNEXE 1: Red Additives: additives for which scientific reports indicate potential carcinogenicity or involvement in serious diseases,
with the following columns:
- Additif: None
- Code: None
--------


--------
Index ID: 83cdb134-0483-4555-872f-d35ede6e9711
--------


--------
The table lists various food additives along with their corresponding codes, specifically highlighting those with potential carcinogenicity or involvement in serious diseases.,
with the following table title:
ANNEXE 1: Red Additives: additives for which scientific reports indicate potential carcinogenicity or involvement in serious diseases,
with the following columns:
- Additif: None
- Code: None

| A

### Criteria Retriever


In [115]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = """You are an expert criterion extractor.
The following text contains element of a convention that needs to be applied. It can be a chunk of text or a table summary.

Text:
{context}

Please:
1. Identify and list all checkable criteria mentioned in the text.
2. If it is a table summary, extract one key criterion that links to the table, specifying that certain aspects must comply with the table's details. Note that the supplier is not responsible for creating the table so do not add criteria such as "- Suppliers must verify that the list of food additives includes their respective codes.
". Suppliers must, but must ensure that the product complies with the table's requirements.
    example : "Suppliers must verify the all the Ingredients are compliant to the 'Specific Requirements for Product Ranges' table", "Suppliers must check if additives are 'Additifs oranges'. Note that you cannot be compliant to a list".
3. If the text or the table is not relevant for criteria checking (introduction, glossary, updates list, etc.), state that no criteria are present with "PASS".
Response format is a list of criterion. All the criterion must start with :
- Les fournisseurs doivent ...

4. Note that suppliers will only give you a list of ingredients and additives present in their product.
5. If it is necessary to refer to a table add {{check_tables}}
6. The criterion have to be written in French.


Complete: 
- 
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context"],
)
llm = ChatOpenAI(model="gpt-4o", temperature = 0.1)
llm_chain = prompt | llm 

In [116]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = """You are an expert criterion extractor.
The following text contains element of a convention that needs to be applied. It can be a chunk of text or a table summary.

Text:
{context}

Please:
1. Identify and list all checkable criteria mentioned in the text.
2. If the context is a table, write a sentence for every lines of the table. The sentence must be a criteria that the supplier must check. If the line is not a criteria, write "PASS".
3. If the text or the table is not relevant for criteria checking (introduction, glossary, document updates, document versions etc.), state that no criteria are present with "PASS".
Response format is a list of criterion. All the criterion must start with :
- Les fournisseurs doivent ...

4. Note that suppliers will only give you a list of ingredients and additives present in their product.
5. If it is necessary to refer to a table add {{check_tables}}
6. Limit the number of criteria to the most important ones.
7. The criterion have to be written in French.


Complete (without repeting the prompt): 
- 
"""

template = """You are an expert criterion extractor.
The following text contains elements of a "Charte Produits/emballage" (Product/Packaging Charter) that needs to be applied. It can be a chunk of text or a table summary.
Text:
{context}
Please:

Identify and list all checkable criteria mentioned in the text, focusing on ingredient categories and their compliance with the charter.
If the text or table is not relevant for criteria checking (e.g., introduction, glossary, document updates, document versions), state that no criteria are present by writing "PASS".
Write the criteria in the same language as the input text.
Start each criterion with a phrase indicating what suppliers must do, such as "Les fournisseurs doivent ..." (Suppliers must ...) for French, or an equivalent phrase in the appropriate language.
Focus on the most important criteria based on the context, but do not exceed a reasonable number of criteria.
Exclude criteria related to document versions, updates, or other non-essential information.

While writing the criteria, consider the following:
- The supplier will provide a list of ingredients and additives present in their product.
- The criteria should be clear and actionable, providing specific instructions for suppliers to follow.
- If necessary, refer to the table provided in the text for additional details.
- The criteria have to be really objective and clear. Do not include subjective criteria or criteria that suggest realising an action for the supplier.
- Next to the criteria, explain your reasoning and why it is really important.
- Limit the criteria to the most important ones.
- Avoid general criterias like "Suppliers must verfy there is no gluten in no gluten products". Be specific and clear.
- Suppose the supplier already gave all the needed document (so don't add it in the criterion)

Complete (without repeating the prompt):

-
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context"],
)
llm = ChatOpenAI(model="gpt-4o", temperature = 0.1)
llm_chain = prompt | llm 

In [117]:
from llama_index.core.schema import IndexNode, TextNode, MetadataMode
# for each Node, extract criterias from the text or summary of the table
llm_results = []
for node in base_nodes_charte:
    if isinstance(node, IndexNode):
        context = node.get_content(metadata_mode=MetadataMode.ALL)
        
    else:
        context = node.get_content()

    llm_result = llm_chain.invoke({"context": context})
    print(llm_result.content)
    print()
    llm_results.append(llm_result.content)

PASS

PASS

- Les fournisseurs doivent garantir que les produits portant l'allégation « sans gluten » ne contiennent aucune trace de gluten. (Raison : La conformité aux allégations « sans gluten » est cruciale pour la sécurité des consommateurs souffrant de la maladie cœliaque ou d'une sensibilité au gluten.)

- Les fournisseurs doivent s'assurer que les produits issus de l’agriculture biologique respectent les normes et certifications biologiques en vigueur. (Raison : Les produits biologiques doivent répondre à des standards spécifiques pour garantir leur qualité et leur conformité aux attentes des consommateurs.)

- Les fournisseurs doivent éviter l'utilisation des additifs répertoriés dans l'ANNEXE I : Additifs rouges, en raison de leur potentielle cancérogénicité ou implication dans des pathologies lourdes. (Raison : La santé des consommateurs est prioritaire, et l'utilisation d'additifs potentiellement dangereux doit être strictement contrôlée.)

- Les fournisseurs doivent limiter

In [118]:
prompt = PromptTemplate(
    template="Rewrite the exact same list but delete repetitive criterion in the text and group them by sub categories : {context}",
    input_variables=["context"],
)
llm_chain = prompt | llm 

llm_results_cleaned = llm_chain.invoke({"context": "\n".join(llm_results)})


In [119]:
print(llm_results_cleaned.content)

### Sécurité des Additifs et Ingrédients

#### Additifs
- **Additifs Rouges (ANNEXE I)**
  - Les fournisseurs doivent éviter l'utilisation des additifs répertoriés en raison de leur potentielle cancérogénicité ou implication dans des pathologies lourdes.
  - Les fournisseurs doivent s'assurer que leurs produits ne contiennent pas d'additifs rouges répertoriés comme potentiellement cancérogènes ou impliqués dans des pathologies lourdes selon les rapports scientifiques.
  - Les fournisseurs doivent fournir une liste complète des additifs présents dans leurs produits, en incluant les codes correspondants pour chaque additif.
  - Les fournisseurs doivent vérifier et confirmer que les additifs utilisés dans leurs produits ne sont pas associés à des risques de cancer ou à des maladies graves, comme indiqué dans les rapports scientifiques.
  - *Raison : La santé des consommateurs est prioritaire, et l'utilisation d'additifs potentiellement dangereux doit être strictement contrôlée.*

- **Addi

In [130]:
ref_criterion = llm_results_cleaned.content

### Excel (target doc) retriever


In [120]:
#read xlsx file : 
import pandas as pd
df = pd.read_excel("./docs/example_fournisseur.xlsx")


In [121]:
xls = pd.ExcelFile("./docs/example_fournisseur.xlsx")
sheets = pd.read_excel(xls, 'Fiche recette')

In [122]:
def table_to_text(df):
    # Initialize an empty list to store the text rows
    text_rows = []

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Combine all columns into a single string separated by " | "
        row_text = " | ".join(str(value) for value in row.values if pd.notna(value))

        # Append the combined string to the list
        if row_text:
            text_rows.append("|" + row_text + "|")

    # Join all rows into a single string separated by newlines
    #result_text = "\n".join(text_rows)

    return text_rows

In [123]:
sheets

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Fiche recette,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,► Recette,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,Merci de détailler ci-dessous la composition exacte du produit tel qu’il est proposé à ARYZTA. La recette doit être découpée par semi-fini et les % de matières premières exprimés par semi-fini. Les ingrédients des MP composées sont à détailler également. Un exemple de déclaration est proposé dans l'onglet EXEMPLE DECLARATION RECETTE.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Type,Dénomination Semi-Fini,% SF,Dénomination Matière Première,% MP,Détail ingrédients si Matière première composée,NaN,Pays d'origine / Zone FAO \n(pour les viandes et les poissons),Informations supplémentaires
8,Semi-Fini,330200 MOUSSE FRAMBOISE MOINS SUCRE,41.5,NaN,NaN,NaN,NaN,NaN,NaN
9,Matière première,NaN,NaN,1120028 FRAMBOISE PUREE SUCREE 10pourcent,34.982509,NaN,NaN,Serbie-Bosnie&Herzegovine,NaN


In [124]:
target_text = table_to_text(sheets)

In [125]:
target_text = "\n".join(target_text)


In [126]:
prompt = PromptTemplate(
    template="""You are a question generator to verify important information in a table. Here are the reference criterion : 
    {ref_criterion}
    I want you to generate questions for each row of the following tables :
     -  The questions should be relevant to the information in the row. Be as specific as possible. Limit the questions to the most important ones.
     -  The questions are adressed to the reference document to verify facts.
     - Question exemple : "Est ce que cet ingrédients est autorisés : ... "?
    here is the table : 
    {target_text}
    """,
    input_variables=["ref_criterion", "target_text"],
)

In [173]:
ref_criterion = """ 
- Les fournisseurs doivent vérifier que tous les ingrédients sont conformes à la table 'Spécificités des gammes de produits' pour les 3 types de produits.
- Les fournisseurs doivent vérifier si les additifs sont oranges, verts ou rouges.
- Les fournisseurs doivent voir selon leur type, si les additifs sont conformes à la table 'Spécificités des gammes de produits' pour les 3 types de produits"""

In [178]:
prompt = PromptTemplate(
    template="""You are a question generator to verify important information in a table. Here are the reference criterion : 
    {ref_criterion}
    I want you to generate questions for each row of the following tables :
     -  The questions should be relevant to the information in the row. Be as specific as possible. Limit the questions to the most important ones.
     -  The questions are adressed to the reference document not to the supplier (the supplier will only provide a list of ingredients and additives present in their product).
     - Question exemple : "Est ce que cet ingrédients est autorisés : ... "?, "De quel type est l'additifs ..., est il autorisé ?"
     - Put the entire table line in the question and split in sub questions when needed (multiple sub ingredients for exemple).
     - Note that Additives should be classified as green, orange or red.
     - Do not format the response, just return the questions.
    here is the table : 
    {target_text}
    """,
    input_variables=["ref_criterion", "target_text"],
)

In [179]:
llm = ChatOpenAI(model="gpt-4o", temperature = 0.1)
llm_chain = prompt | llm 

questions = llm_chain.invoke({"ref_criterion":ref_criterion, "target_text": target_text })

In [180]:
question_list = str(questions.content).split("\n")

In [181]:
question_list

['1. Est-ce que cet ingrédient est autorisé : 1120028 FRAMBOISE PUREE SUCREE 10pourcent (Serbie-Bosnie&Herzegovine) ?',
 '2. Est-ce que cet ingrédient est autorisé : 1050004 EAU (France) ?',
 '3. Est-ce que cet ingrédient est autorisé : 1020001 SUCRE CRISTAL (UE) ?',
 '4. Est-ce que ces ingrédients sont autorisés : LAMEQUICK CE RSPO (UE des 27) ? ',
 '   - sirop de glucose',
 '   - huile de palmiste',
 '   - émulsifiant : E472a (acetic acid esters of mono- and diglycerides of fatty acids) (étiquettage : émulsifiant : E472a (LAIT), le reste : support non déclarable - palme RSPO)',
 '   - protéine de lait',
 "5. De quel type est l'additif E472a (acetic acid esters of mono- and diglycerides of fatty acids) dans LAMEQUICK CE RSPO, est-il autorisé ?",
 '6. Est-ce que ces ingrédients sont autorisés : 1030011 CREME 35% UHT 10L DDJ (UE) ?',
 '   - crème de LAIT',
 '   - stabilisant : E407',
 "7. De quel type est l'additif E407 dans CREME 35% UHT 10L DDJ, est-il autorisé ?",
 '8. Est-ce que cet

### Recursive Retriever

In [30]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import VectorStoreIndex

In [37]:
# construct top-level vector index + query engine
vector_index = VectorStoreIndex(base_nodes_charte)
vector_retriever = vector_index.as_retriever(similarity_top_k=10)
vector_query_engine = vector_index.as_query_engine(similarity_top_k=10)

In [ ]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
)
extractors = [
    SummaryExtractor(summaries=["self"], show_progress=True),
    QuestionsAnsweredExtractor(questions=5, show_progress=True),
]

In [38]:
from llama_index.core.retrievers import RecursiveRetriever

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    node_dict=node_mappings_charte,
    verbose=True,
)
query_engine = RetrieverQueryEngine.from_args(recursive_retriever)

In [64]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
from llama_index.core.postprocessor import LLMRerank


def get_retrieved_nodes(
    query_str, retriever, vector_top_k=10, reranker_top_n=3, with_reranker=False
):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        # configure reranker
        reranker = LLMRerank(
            choice_batch_size=5,
            top_n=reranker_top_n,
        )
        retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )

    return retrieved_nodes

def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "<br>")))

def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

In [108]:
from llama_index.postprocessor.cohere_rerank import CohereRerank


cohere_rerank = CohereRerank(top_n=10)
query_engine = vector_index.as_query_engine(
    similarity_top_k=15,
    node_postprocessors=[cohere_rerank],
)
response = query_engine.query(
    "Quel est le type de l'additif E472a ?",
)

In [109]:
response

Response(response="L'additif E472a n'est pas spécifiquement mentionné dans les annexes fournies, donc je ne peux pas déterminer son type à partir des informations disponibles.", source_nodes=[NodeWithScore(node=TextNode(id_='a9016b50-b17e-483a-ab89-9a84f46b89c2', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0215dbc4-391b-4869-bbc6-a6cc46b034e1', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='f2dc49bb5b95e8c48cc6ad228ccb68912d27445a7cd7e638309114298813af06'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='92cda115-280e-4836-8af2-6d4b4e6f8e34', node_type=<ObjectType.INDEX: '3'>, metadata={'col_schema': 'Column: Additif\nType: string\nSummary: None\n\nColumn: Code\nType: string\nSummary: None'}, hash='359949570ab56ce489f2bef5586ad51d172303db9e0d0c7c99acd3f6761dff56')}, text='ANNEXE III : Additifs verts : additifs identifiés à ce jour comme non dangereux

In [110]:
retrieved_chunk = query_engine.retrieve(QueryBundle("Est ce que l'additif E472a est accépté dans les produits ?"))

In [116]:
retrieved_chunk[4].node.get_text()

'This table lists food additives with their corresponding codes, specifically highlighting those for which scientific reports are contradictory.,\nwith the following table title:\nANNEXE II : Additifs oranges : additifs pour lesquels les rapports scientifiques sont contradictoires,\nwith the following columns:\n- Additif: None\n- Code: None\n- Additif: None\n- Code: None\n'

In [85]:
import pandas as pd
from IPython.display import display, HTML

new_nodes = get_retrieved_nodes(
    "Est ce que l'additif E472a est accépté dans les produits ?",
    retriever= recursive_retriever,
    vector_top_k=15,
    # reranker_top_n=10,
    # with_reranker=True,
)
visualize_retrieved_nodes(new_nodes)

Retrieving with query id None: Est ce que l'additif E472a est accépté dans les produits ?
Retrieving text node: Table des matières

1. Exigence recette 2
2. Produits soumis à certification ou allégations 3
   2.1. Produits « sans gluten » 3
   2.2. Produits issus de l’agriculture biologique 3
3. Exigences générales relatives au fournisseur 4
4. Exigences relatives aux sites de production 4
5. Traçabilité 4
6. Suivi analytique 5
   6.1. Suivi microbiologique 5
   6.2. Suivi nutritionnel 5
   6.3. Suivi organoleptique 5
7. Non conformités 5
8. Gestion de crise Coup de Pates 6

ANNEXE I : Additifs rouges : additifs pour lesquels les rapports scientifiques rapportent une potentielle cancérogénicité ou une implication dans les pathologies lourdes 7

ANNEXE II : Additifs oranges : additifs pour lesquels les rapports scientifiques sont contradictoires 10

ANNEXE III : Additifs verts : additifs identifiés à ce jour comme non dangereux pour la santé 11

ANNEXE IV : Ingrédients controversés 12
R

In [39]:
from llama_index.core.response.notebook_utils import display_source_node

nodes = recursive_retriever.retrieve("Est ce que l'additif E472a est autorisé dans les produits hauts de gamme ?")
for node in nodes:
    display_source_node(node, source_length=2000)

Retrieving with query id None: Est ce que l'additif E472a est autorisé dans les produits hauts de gamme ?
Retrieving text node: Table des matières

1. Exigence recette 2
2. Produits soumis à certification ou allégations 3
   2.1. Produits « sans gluten » 3
   2.2. Produits issus de l’agriculture biologique 3
3. Exigences générales relatives au fournisseur 4
4. Exigences relatives aux sites de production 4
5. Traçabilité 4
6. Suivi analytique 5
   6.1. Suivi microbiologique 5
   6.2. Suivi nutritionnel 5
   6.3. Suivi organoleptique 5
7. Non conformités 5
8. Gestion de crise Coup de Pates 6

ANNEXE I : Additifs rouges : additifs pour lesquels les rapports scientifiques rapportent une potentielle cancérogénicité ou une implication dans les pathologies lourdes 7

ANNEXE II : Additifs oranges : additifs pour lesquels les rapports scientifiques sont contradictoires 10

ANNEXE III : Additifs verts : additifs identifiés à ce jour comme non dangereux pour la santé 11

ANNEXE IV : Ingrédients c

**Node ID:** ed06869d-f5bd-49cd-91ea-447161ea150d<br>**Similarity:** 0.508949073332592<br>**Text:** Table des matières

1. Exigence recette 2
2. Produits soumis à certification ou allégations 3
   2.1. Produits « sans gluten » 3
   2.2. Produits issus de l’agriculture biologique 3
3. Exigences générales relatives au fournisseur 4
4. Exigences relatives aux sites de production 4
5. Traçabilité 4
6. Suivi analytique 5
   6.1. Suivi microbiologique 5
   6.2. Suivi nutritionnel 5
   6.3. Suivi organoleptique 5
7. Non conformités 5
8. Gestion de crise Coup de Pates 6

ANNEXE I : Additifs rouges : additifs pour lesquels les rapports scientifiques rapportent une potentielle cancérogénicité ou une implication dans les pathologies lourdes 7

ANNEXE II : Additifs oranges : additifs pour lesquels les rapports scientifiques sont contradictoires 10

ANNEXE III : Additifs verts : additifs identifiés à ce jour comme non dangereux pour la santé 11

ANNEXE IV : Ingrédients controversés 12<br>

**Node ID:** ae1ee1b4-e905-4399-8fc5-f64c413b77bc<br>**Similarity:** 0.4840262840056945<br>**Text:** L’ensemble de ces critères est applicable à tous les produits vendus par Coup de Pates. Des dérogations peuvent être accordées au cas par cas, sur justificatifs fournis par le fournisseur et après validation par la direction qualité Coup de Pates.<br>

**Node ID:** d82b3c66-05b9-4753-a0e7-1553a5118291<br>**Similarity:** 0.46541714578958293<br>**Text:** ANNEXE IV : Ingrédients controversés : ingrédients faisant l’objet de rapports scientifiques controversés et/ou perçus négativement par le consommateur.<br>

**Node ID:** 83cdb134-0483-4555-872f-d35ede6e9711<br>**Similarity:** 0.465071086757835<br>**Text:** The table lists various food additives along with their corresponding codes, specifically highlighting those with potential carcinogenicity or involvement in serious diseases.,
with the following table title:
ANNEXE 1: Red Additives: additives for which scientific reports indicate potential carcinogenicity or involvement in serious diseases,
with the following columns:
- Additif: None
- Code: None

| Additif                          | Code  | Additif                                | Code  |
|----------------------------------|-------|----------------------------------------|-------|
| Tartrazine                       | E 102 | Acide propionique                      | E 280 |
| Jaune de quinoléine              | E 104 | Propionate de sodium                   | E 281 |
| Sunset Yellow FCF/Jaune orange S | E 110 | Propionate de potassium                | E 283 |
| Azorubine, carmoisine            | E 122 | Acide borique                          | E 284 |
| Amarante                         | E 123 | Tétraborate de sodium (borax)          | E 285 |
| Ponceau 4R, rouge cochenille A   | E 124 | Acide fumarique                        | E 297 |
| Erythrosine                      | E 127 | Gamma-tocophérol                       | E 308 |
| Rouge allura AC                  | E 129 | Delta-tocophérol                       | E 309 |
| Indigotine, carmin d’indigo      | E 132 | Gallate de propyle                     | E 310 |
| Bleu brillant FCF                | E 133 | Acide érythorbique                     | E 315 |
| Vert S                           | E 142 | Butylhydro-quinone tertiaire (BHQ)     | E 319 |
| Caramel ammoniacal               | E 150c| Butylhydroxy-anisol (BHA)              | E 320 |
| Caramel au sulfite d’ammonium    | E 150d| Butylhydroxy-toluène (BHT)             | E 321 |
| Noir brillant PN                 | E 151 | Tartrates de sodium                    | E 335 |
| Brun HT                          | E 155 | Tartrate double de sodium et de potassium | E ...<br>

...

**Node ID:** 6f2d928f-19c1-4f2e-ac68-298a74715eda<br>**Similarity:** 0.45520509873185944<br>**Text:** 1. Exigence recette

Pour le développement de nos produits, nous souhaitons favoriser une offre saine avec des recettes simples (sans colorant, sans arôme, sans conservateur), avec des ingrédients de qualité, en favorisant des produits locaux et labellisés.

Le fournisseur s’engage à respecter la réglementation européenne et nationale ainsi que les codes d’usages professionnels applicables aux produits surgelés vendus à Coup de Pates.

Pour les produits commercialisés sous une marque appartenant à Coup de Pates, le fournisseur s’engage également à respecter les exigences spécifiques de cette même marque.

Dans ce cas, nos exigences recettes sont spécifiques à trois niveaux gammes : Entrée de gamme, Cœur de gamme, Haut de gamme.

Pour les produits développés en réponse à des demandes spécifiques de nos clients, il vous sera également demandé de prendre leurs exigences en considération.<br>

**Node ID:** 142d9f89-f960-48df-9345-184087b26ef2<br>**Similarity:** 0.4497618277978406<br>**Text:** This table lists food additives with their corresponding codes, specifically highlighting those for which scientific reports are contradictory.,
with the following table title:
ANNEXE II : Additifs oranges : additifs pour lesquels les rapports scientifiques sont contradictoires,
with the following columns:
- Additif: None
- Code: None
- Additif: None
- Code: None

| Additif | Code | Additif .1| Code .1|
|---|---|---|---|
| Acide carminique, carmins | E 120 | Esters lactiques des mono- et diglycérides d’acides gras | E 472b |
| Bleu patenté V | E 131 | Esters citriques des mono- et diglycérides d’acides gras | E 472c |
| Caramel de sulfite caustique | E 150b | Esters tartriques des mono- et diglycérides d’acides gras | E 472d |
| Or | E 175 | Esters monoacétyltartriques et diacétyltartriques des mono- et diglycérides d’acides gras | E 472e |
| Acide benzoïque | E 210 | Esters mixtes acétiques et tartriques des mono- et diglycérides d’acides gras | E 472f |
| Benzoate de sodium | E 211 | Sucroesters d’acides gras | E 473 |
| Anhydride sulfureux | E 220 | Esters polyglycériques d’acides gras | E 475 |
| Sulfite de sodium | E 221 | Esters de propane-1,2-diol d’acides gras | E 477 |
| Sulfite acide de sodium | E 222 | Stéaroyl-2-lactylate de sodium | E 481 |
| Disulfite de sodium | E 223 | Stéaroyl-2-lactylate de calcium | E 482 |
| Disulfite de potassium | E 224 | Tartrate de stéaryle | E 483 |
| Sulfite de calcium | E 226 | Diméthylpolysiloxane | E 900 |
| Sulfite acide de calcium | E 227 | Advantame | E 969 |
| Sulfite acide de potassium | E 228 | Extraits de quillaia | E 999 |
| Natamycine | E 235 | Lysozyme | E 1105 |
| Éthylène-diamine-tétra-acétate de calcium disodium (calcium disodium EDTA) | E 385 | Amidon oxydé | E 1404 |
| Alginate de propane-1,2-diol | E 405 | Phosphate de monoamidon | E 1410 |
| **Algues Euchema transformées** | **E 407a** | **Phosphate de diamidon** | **E 1412** |
|---------------------------------|------------|--------------------------...<br>

**Node ID:** 8fde517a-55b7-4d9c-9507-ee0aff5953fd<br>**Similarity:** 0.44925682612916445<br>**Text:** ANNEXE 1: Additifs rouges : additifs pour lesquels les rapports scientifiques rapportent une potentielle cancérogénicité ou une implication dans les pathologies lourdes<br>

**Node ID:** 5e96cb0e-e733-4977-aff2-77177d829b66<br>**Similarity:** 0.4412167468027831<br>**Text:** This table lists controversial ingredients, detailing the reasons for their controversy or negative perception by consumers.,
with the following table title:
ANNEXE IV : Ingrédients controversés,
with the following columns:
- Ingrédient: None
- Motif: None

| Ingrédient                | Motif                                                                 |
|---|---|
| Sirop de glucose-fructose | Niveau de transformation élevé + manque de transparence sur le niveau de sucre présent dans le produit |
| Maltodextrine             | Ingrédient sans intérêt nutritionnel et organoleptique                |
| Huile de coco/coprah      | Contient 80% d’acides gras saturés dont l’excès augmente le risque de maladies cardiovasculaires |
| Sirop de maïs             | Niveau de transformation élevé + manque de transparence sur le niveau de sucre présent dans le produit |<br>

**Node ID:** 14834fc2-def0-4082-a937-6d35366e0dd1<br>**Similarity:** 0.43976527828820633<br>**Text:** The table outlines the ingredient and processing restrictions for different product ranges (Entry-level, Mid-range, High-end) within a company. It specifies which substances are prohibited or to be avoided across these ranges, ensuring compliance with specific quality and safety standards.,
with the following table title:
Product Compliance Standards by Range,
with the following columns:
- Caractéristiques: None
- Entrée de gamme: None
- Cœur de gamme: None
- Haut de Gamme: None

| Caractéristiques | Entrée de gamme | Cœur de gamme | Haut de Gamme |
|---|---|---|---|
| Ingrédients soumis à déclaration OGM | INTERDIT | INTERDIT | INTERDIT |
| Traitement par ionisation | INTERDIT | INTERDIT | INTERDIT |
| Colorants azoïques (E102, E104, E110, E122, E124, E129) | INTERDIT | INTERDIT | INTERDIT |
| Nanoparticules (E170, E171, E172, E174, E152, E341, E551 et E552) | INTERDIT | INTERDIT | INTERDIT |
| Glutamates et exhausteurs de goût | INTERDIT | INTERDIT | INTERDIT |
| Œufs de poules élevées en cage | INTERDIT | INTERDIT | INTERDIT |
| Matières grasses partiellement hydrogénées | INTERDIT | INTERDIT | INTERDIT |
| Acides gras trans non naturellement présents | INTERDIT | INTERDIT | INTERDIT |
| Édulcorants de synthèse | INTERDIT | INTERDIT | INTERDIT |
| Viande Séparée Mécaniquement - VSM | INTERDIT | INTERDIT | INTERDIT |
| Cacao non certifié durable | * INTERDIT pour tous les NPD et plan action pour remplacer le cacao non certifié dans l’existant. | INTERDIT | INTERDIT |
| Gélatine porcine | INTERDIT | INTERDIT | INTERDIT |
| Gélatine animale – (autre que porcine) | A ÉVITER | INTERDIT (tolérance dans les pâtisseries) | INTERDIT (tolérance dans les pâtisseries) |
| Huile de palme + palmiste non RSPO | * INTERDIT pour tous les NPD et plan action pour retirer dans l’existant - (tolérée dans supports d’additifs) - En aucun cas, l’huile de palme non RSPO ne pourra être substituée par de l’huile de coprah ou coco. | INTERDIT | INTERDIT |
| Huile de palme + palmiste RSPO...<br>

In [88]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-4o")

query_engine = RetrieverQueryEngine.from_args(recursive_retriever, llm=llm)

In [90]:
response = query_engine.query("Est ce que l'additif E472a est autorisé dans les produits hauts de gamme, prends en compte tous les éléments?")
print(str(response))

Retrieving with query id None: Est ce que l'additif E472a est autorisé dans les produits hauts de gamme, prends en compte tous les éléments?
Retrieving text node: Table des matières

1. Exigence recette 2
2. Produits soumis à certification ou allégations 3
   2.1. Produits « sans gluten » 3
   2.2. Produits issus de l’agriculture biologique 3
3. Exigences générales relatives au fournisseur 4
4. Exigences relatives aux sites de production 4
5. Traçabilité 4
6. Suivi analytique 5
   6.1. Suivi microbiologique 5
   6.2. Suivi nutritionnel 5
   6.3. Suivi organoleptique 5
7. Non conformités 5
8. Gestion de crise Coup de Pates 6

ANNEXE I : Additifs rouges : additifs pour lesquels les rapports scientifiques rapportent une potentielle cancérogénicité ou une implication dans les pathologies lourdes 7

ANNEXE II : Additifs oranges : additifs pour lesquels les rapports scientifiques sont contradictoires 10

ANNEXE III : Additifs verts : additifs identifiés à ce jour comme non dangereux pour la 

In [44]:
print(str(response))

Le E472a n'est pas mentionné spécifiquement dans les restrictions pour les produits hauts de gamme. Par conséquent, il n'est pas explicitement interdit ou à éviter selon les informations fournies.


In [14]:
response = query_engine.query("Est ce que la Cire de carnauba est un additif de type orange? Sinon, de quel type s'agit il ? Quelle est son code E ?")
print(str(response))

Retrieving with query id None: Est ce que la Cire de carnauba est un additif de type orange? Sinon, de quel type s'agit il ? Quelle est son code E ?
Retrieved node with id, entering: 142d9f89-f960-48df-9345-184087b26ef2
Retrieving with query id 142d9f89-f960-48df-9345-184087b26ef2: Est ce que la Cire de carnauba est un additif de type orange? Sinon, de quel type s'agit il ? Quelle est son code E ?
Retrieved node with id, entering: 83cdb134-0483-4555-872f-d35ede6e9711
Retrieving with query id 83cdb134-0483-4555-872f-d35ede6e9711: Est ce que la Cire de carnauba est un additif de type orange? Sinon, de quel type s'agit il ? Quelle est son code E ?
Retrieving text node: ANNEXE II : Additifs oranges : additifs pour lesquels les rapports scientifiques sont contradictoires
Retrieved node with id, entering: 76adb659-e69e-4acf-bd42-d3176d389ccc
Retrieving with query id 76adb659-e69e-4acf-bd42-d3176d389ccc: Est ce que la Cire de carnauba est un additif de type orange? Sinon, de quel type s'agit 

In [15]:
response = query_engine.query("Est ce que la Maltodextrine est un ingrédient controversé ? et pour quelle raison ?")
print(str(response))

Retrieving with query id None: Est ce que la Maltodextrine est un ingrédient controversé ? et pour quelle raison ?
Retrieved node with id, entering: 5e96cb0e-e733-4977-aff2-77177d829b66
Retrieving with query id 5e96cb0e-e733-4977-aff2-77177d829b66: Est ce que la Maltodextrine est un ingrédient controversé ? et pour quelle raison ?
Retrieving text node: ANNEXE IV : Ingrédients controversés : ingrédients faisant l’objet de rapports scientifiques controversés et/ou perçus négativement par le consommateur.
Retrieving text node: Table des matières

1. Exigence recette 2
2. Produits soumis à certification ou allégations 3
   2.1. Produits « sans gluten » 3
   2.2. Produits issus de l’agriculture biologique 3
3. Exigences générales relatives au fournisseur 4
4. Exigences relatives aux sites de production 4
5. Traçabilité 4
6. Suivi analytique 5
   6.1. Suivi microbiologique 5
   6.2. Suivi nutritionnel 5
   6.3. Suivi organoleptique 5
7. Non conformités 5
8. Gestion de crise Coup de Pates 6



In [19]:
# compare against the baseline retriever
response = vector_query_engine.query("Est ce que la Cire de carnauba est un additif de type orange? Quelle est son code E ?")
print(str(response))

Pour déterminer si l'additif E 903 est de type orange, il faudrait vérifier s'il figure dans la liste des additifs oranges mentionnée dans l'ANNEXE II. Cette annexe regroupe les additifs pour lesquels les rapports scientifiques sont contradictoires.


## Document Comparaison using this llama parse ref

In [117]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings

embed_model = OpenAIEmbedding(model="text-embedding-3-small")
llm = OpenAI(model="gpt-4o")

Settings.llm = llm
Settings.embed_model = embed_model

In [118]:
from llama_index.core.node_parser import MarkdownElementNodeParser

node_parser = MarkdownElementNodeParser(
    llm=OpenAI(model="gpt-4o"), num_workers=8
)

In [119]:
import pickle
from llama_index.postprocessor.flag_embedding_reranker import (
    FlagEmbeddingReranker,
)

reranker = FlagEmbeddingReranker(
    top_n=10,
    model="BAAI/bge-reranker-large",
)


def create_query_engine_over_doc(docs, nodes_save_path=None):
    """Big function to go from document path -> recursive retriever."""
    if nodes_save_path is not None and os.path.exists(nodes_save_path):
        raw_nodes = pickle.load(open(nodes_save_path, "rb"))
    else:
        raw_nodes = node_parser.get_nodes_from_documents(docs)
        if nodes_save_path is not None:
            pickle.dump(raw_nodes, open(nodes_save_path, "wb"))

    base_nodes, objects = node_parser.get_nodes_and_objects(raw_nodes)

    ### Construct Retrievers
    # construct top-level vector index + query engine
    vector_index = VectorStoreIndex(nodes=base_nodes + objects)
    query_engine = vector_index.as_query_engine(
        similarity_top_k=15, node_postprocessors=[reranker]
    )
    return query_engine, base_nodes

/Users/chloed./Documents/quivr/quivr/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [121]:
query_engine, nodes = create_query_engine_over_doc(
   charte, nodes_save_path="docs/charte.pkl"
)

In [130]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine


# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="charte_produit",
            description=("Donne des informations sur la charte produit à respecter. Donne une liste d'ingredient interdit/ toléré selon si le produit est Haut de gamme, Milieu de gamme ou entrée de gamme. Donne une liste des additifs selon leurs type : vert, rouge ou orange"),
        ),
    )
]

sub_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    llm=llm,
    use_async=True,
)

In [131]:
response = sub_query_engine.query(
   "Est ce que le Diglutamate de magnésium est autorisé ?"
)

Generated 4 sub questions.
[charte_produit] Q: Le Diglutamate de magnésium est-il un ingrédient interdit, toléré ou autorisé pour les produits Haut de gamme ?
[charte_produit] Q: Le Diglutamate de magnésium est-il un ingrédient interdit, toléré ou autorisé pour les produits Milieu de gamme ?
[charte_produit] Q: Le Diglutamate de magnésium est-il un ingrédient interdit, toléré ou autorisé pour les produits Entrée de gamme ?
[charte_produit] Q: Le Diglutamate de magnésium est-il classé comme un additif vert, rouge ou orange ?
[charte_produit] A: Le Diglutamate de magnésium est interdit pour les produits Milieu de gamme.
[charte_produit] A: Le Diglutamate de magnésium est classé comme un additif rouge.
[charte_produit] A: Le Diglutamate de magnésium est un ingrédient interdit pour les produits Haut de gamme.
[charte_produit] A: Le Diglutamate de magnésium n'est pas mentionné dans les listes d'ingrédients interdits, tolérés ou autorisés pour les produits Entrée de gamme.


In [133]:
response.response

"Le Diglutamate de magnésium est interdit pour les produits Haut de gamme et Milieu de gamme, et il n'est pas mentionné pour les produits Entrée de gamme."

In [124]:
print(str(response))

L'additif E472a n'est pas autorisé dans les produits hauts de gamme.
